In [1]:
'''Code for fine-tuning Inception V3 for a new task.

Start with Inception V3 network, not including last fully connected layers.

Train a simple fully connected layer on top of these.


'''

import numpy as np
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Dropout
import inception_v3 as inception

N_CLASSES = 2
IMSIZE = (299, 299)

# TO DO:: Replace these with paths to the downloaded data.
# Training directory
train_dir = '/mnt/e/data/catdog/train'
# Testing directory
test_dir = '/mnt/e/data/catdog/validation'


# Start with an Inception V3 model, not including the final softmax layer.
base_model = inception.InceptionV3(weights='imagenet')
print ('Loaded Inception model')

Using TensorFlow backend.
/mnt/d/git/numbers/project/code/inception_v3.py:57: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", name=None, strides=(2, 2), padding="valid")`
  name=conv_name)(x)
/mnt/d/git/numbers/project/code/inception_v3.py:57: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", name=None, strides=(1, 1), padding="valid")`
  name=conv_name)(x)
/mnt/d/git/numbers/project/code/inception_v3.py:57: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", name=None, strides=(1, 1), padding="same")`
  name=conv_name)(x)
/mnt/d/git/numbers/project/code/inception_v3.py:57: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(80, (1, 1), activation="relu", name=None, strides=(1, 1), padding="valid")`
  name=conv_name)(x)
/mnt/d/git/numbers/project/code/inception_v3.py:57: UserWarning: Update your `Conv2D` call to the Keras 2 API: 

Loaded Inception model


In [2]:
for layer in base_model.layers:
    print(layer.output)

Tensor("input_1:0", shape=(?, 299, 299, 3), dtype=float32)
Tensor("conv2d_1/Relu:0", shape=(?, 149, 149, 32), dtype=float32)
Tensor("batch_normalization_1/cond/Merge:0", shape=(?, 149, 149, 32), dtype=float32)
Tensor("conv2d_2/Relu:0", shape=(?, 147, 147, 32), dtype=float32)
Tensor("batch_normalization_2/cond/Merge:0", shape=(?, 147, 147, 32), dtype=float32)
Tensor("conv2d_3/Relu:0", shape=(?, 147, 147, 64), dtype=float32)
Tensor("batch_normalization_3/cond/Merge:0", shape=(?, 147, 147, 64), dtype=float32)
Tensor("max_pooling2d_1/MaxPool:0", shape=(?, 73, 73, 64), dtype=float32)
Tensor("conv2d_4/Relu:0", shape=(?, 73, 73, 80), dtype=float32)
Tensor("batch_normalization_4/cond/Merge:0", shape=(?, 73, 73, 80), dtype=float32)
Tensor("conv2d_5/Relu:0", shape=(?, 71, 71, 192), dtype=float32)
Tensor("batch_normalization_5/cond/Merge:0", shape=(?, 71, 71, 192), dtype=float32)
Tensor("max_pooling2d_2/MaxPool:0", shape=(?, 35, 35, 192), dtype=float32)
Tensor("conv2d_9/Relu:0", shape=(?, 35, 35,

In [3]:
base_model.get_layer('flatten').input

<tf.Tensor 'avg_pool/AvgPool:0' shape=(?, 1, 1, 2048) dtype=float32>

In [4]:
# Turn off training on base model layers
for layer in base_model.layers:
    layer.trainable = False
k=base_model.get_layer('flatten').output
# Add on new fully connected layers for the output classes.
x = Dense(32, activation='relu')(base_model.get_layer('flatten').output)
x = Dropout(0.5)(x)
predictions = Dense(N_CLASSES, activation='softmax', name='predictions')(x)

model = Model(input=base_model.input, output=predictions)

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])


# Show some debug output
print (model.summary())

print ('Trainable weights')
print (model.trainable_weights)

/home/john/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("pr...)`


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 299, 299, 3)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 149, 149, 32)  896                                          
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 149, 149, 32)  128                                          
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, 147, 147, 32)  9248                                         
___________________________________________________________________________________________

In [11]:
# Data generators for feeding training/testing images to the model.
model.load_weights('catdog_pretrain.h5') 
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        train_dir,  # this is the target directory
        target_size=IMSIZE,  # all images will be resized to 299x299 Inception V3 input
        batch_size=8,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        test_dir,  # this is the target directory
        target_size=IMSIZE,  # all images will be resized to 299x299 Inception V3 input
        batch_size=8,
        class_mode='categorical')

model.fit_generator(
        train_generator,
        samples_per_epoch=32,
        nb_epoch=5,
        validation_data=test_generator,
        verbose=2,
        nb_val_samples=64)
model.save_weights('catdog_pretrain.h5')  # always save your weights after training or during training




# img_path = '../data/sport3/validation/hockey/img_2997.jpg'
# img = image.load_img(img_path, target_size=IMSIZE)
# x = image.img_to_array(img)
# x = np.expand_dims(x, axis=0)

# x = inception.preprocess_input(x)

# preds = model.predict(x)
# print('Predicted:', preds)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


/home/john/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:23: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., verbose=2, steps_per_epoch=4, epochs=5, validation_steps=64)`


Epoch 1/5
204s - loss: 0.1664 - acc: 1.0000 - val_loss: 0.1467 - val_acc: 0.9531
Epoch 2/5
212s - loss: 0.4296 - acc: 0.8438 - val_loss: 0.1218 - val_acc: 0.9648
Epoch 3/5
206s - loss: 0.2424 - acc: 0.9375 - val_loss: 0.1117 - val_acc: 0.9688
Epoch 4/5
199s - loss: 0.2246 - acc: 0.8438 - val_loss: 0.1054 - val_acc: 0.9746
Epoch 5/5
199s - loss: 0.2406 - acc: 0.9062 - val_loss: 0.1041 - val_acc: 0.9668


In [9]:
model.load_weights('catdog_pretrain.h5') 
img_path = '../data/dog1.jpg'
img = image.load_img(img_path, target_size=IMSIZE)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

x = inception.preprocess_input(x)

preds = model.predict(x)
print('Predicted:', preds)

Predicted: [[ 0.36576056  0.63423944]]
